In [1]:
from langchain_openai import ChatOpenAI
from decouple import config


In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model='llama2')

In [4]:
llm.invoke("how can langsmith help with testing?")

'Langsmith is a language model that can assist with testing in various ways:\n\n1. Automated Testing: Langsmith can be used to generate test cases automatically, based on the API documentation of an application. This can save time and reduce the likelihood of human error.\n2. Code Review: Langsmith can analyze code and identify potential issues, such as security vulnerabilities or coding standards violations. It can also provide recommendations for improvement.\n3. Test Data Generation: Langsmith can generate test data automatically, based on the API documentation of an application. This can be particularly useful for testing APIs that require a large amount of test data.\n4. Test Case Optimization: Langsmith can optimize test cases by identifying the most critical test cases and prioritizing them based on risk and complexity.\n5. Defect Prediction: Langsmith can analyze code and identify potential defects, allowing developers to proactively fix issues before they become problems.\n6. 

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a technical documentation writer, I must say that Langsmith is a fantastic tool for testing and quality assurance. Here are some ways in which Langsmith can help with testing:\n\n1. Automated Testing: With Langsmith's powerful natural language processing capabilities, you can create automated tests that can analyze and validate your technical documentation. For example, you can use Langsmith to test for consistency in formatting, layout, and style throughout your documentation.\n2. Content Validation: Langsmith can help ensure that your technical documentation is accurate and up-to-date by validating the content against a set of predefined rules or standards. This can be particularly useful when working with complex technical products or systems, where small errors in documentation can have significant consequences.\n3. User Interface Testing: Langsmith can also help test your user interface (UI) and user experience (UX) design by analyzing the content 

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser


In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is an excellent tool for testing purposes! Here are some ways in which Langsmith can assist with testing:\n\n1. Automated Testing: Langsmith's advanced natural language processing (NLP) capabilities enable it to perform automated testing of technical documents. This means that Langsmith can analyze and evaluate the content of your technical documentation automatically, identifying any errors or inconsistencies.\n2. Consistency Checking: Langsmith can check for consistency in your technical documentation by analyzing your content against a set of predefined rules or patterns. This helps ensure that your documentation is consistent throughout, which is crucial for maintaining accuracy and clarity.\n3. Style Guidance: Langsmith's style guide feature can help you identify and correct any inconsistencies in formatting, syntax, and other stylistic elements across your technical documents. This ensures that your doc

In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [5]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()

In [30]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [31]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [33]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results.'

In [34]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [35]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Based on the provided context, LangSmith can help with testing by providing features such as:

* Evaluation capabilities: LangSmith offers evaluation capabilities that allow you to test and assess the performance of your LLM applications.
* Tracing capabilities: LangSmith provides tracing capabilities that enable you to track the behavior of your LLM applications during testing, allowing you to identify potential issues and optimize their performance.
* Integration with LangChain: LangSmith seamlessly integrates with LangChain, the go-to open source framework for building LLMs, providing a comprehensive platform for testing and evaluating LLM applications.


In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [37]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [39]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

In [40]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [8]:
llm = ChatOpenAI(openai_api_key=config('openai_api_key'))

In [9]:
llm.invoke("how can langsmith help with testing?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [1]:
llm.invoke("how can langsmith help with testing?")

NameError: name 'llm' is not defined